In [1]:
import boto3

In [2]:
s = boto3

In [3]:
ec2sr = s.resource('ec2', region_name='ap-east-1')
ec2cl = boto3.client('ec2', region_name='ap-east-1')

In [4]:
instanceID = 'i-0f013081393ca3e42'
ec2Boot = ec2sr.Instance(instanceID)
print(ec2Boot.instance_id)
print(ec2Boot.instance_type)
print(ec2Boot.state['Name'])
print(ec2Boot.public_ip_address)

i-0f013081393ca3e42
t3.small
running
18.162.52.4


In [5]:
labHostedZoneID = 'Z28ASKRNGI90AH'
labHotsedZone = 'lab.winglokhouse.com'
ec2fqdn= 'whiz.lab.winglokhouse.com'
s1 = boto3
route53cl = s1.client('route53')
dnsResp = route53cl.list_resource_record_sets(HostedZoneId=labHostedZoneID,StartRecordName=ec2fqdn,StartRecordType='A')

In [6]:
dnsRRList = dnsResp['ResourceRecordSets']
dnsRRList

[{'Name': 'whiz.lab.winglokhouse.com.',
  'Type': 'A',
  'TTL': 300,
  'ResourceRecords': [{'Value': '18.162.111.174'}]}]

In [7]:
print(dnsRRList[0]['ResourceRecords'][0]['Value'])

18.162.111.174


In [8]:
ec2Boot.public_ip_address

'18.162.52.4'

In [9]:
dnsChangeAction = 'UPSERT'

In [10]:
dnsChange={
    'Changes': [
        {
            'Action': dnsChangeAction,
            'ResourceRecordSet': {
                'Name': ec2fqdn,
                'Type': 'A',
                'TTL': 300,
                'ResourceRecords': [
                    {
                        'Value': ec2Boot.public_ip_address
                    }
                ]
            }
        }
    ]
}

In [11]:
dnsChange

{'Changes': [{'Action': 'UPSERT',
   'ResourceRecordSet': {'Name': 'whiz.lab.winglokhouse.com',
    'Type': 'A',
    'TTL': 300,
    'ResourceRecords': [{'Value': '18.162.52.4'}]}}]}

In [12]:
dnsResp = route53cl.change_resource_record_sets(HostedZoneId=labHostedZoneID,ChangeBatch=dnsChange)

In [13]:
dnsResp

{'ResponseMetadata': {'RequestId': '29b93fd0-94d2-11e9-a32b-7937884b0ed5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '29b93fd0-94d2-11e9-a32b-7937884b0ed5',
   'content-type': 'text/xml',
   'content-length': '276',
   'date': 'Sat, 22 Jun 2019 09:43:17 GMT'},
  'RetryAttempts': 0},
 'ChangeInfo': {'Id': '/change/C2J4JGMYCYHP4R',
  'Status': 'PENDING',
  'SubmittedAt': datetime.datetime(2019, 6, 22, 9, 43, 17, 589000, tzinfo=tzutc())}}